In [1]:
!pip install timm
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [2]:
import json

import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
# !ls

In [ ]:
!tar -xf videos.tar --directory videos/

In [ ]:
data.head()

,label,youtube_id,time_start,time_end,split
0,abseiling,Z9CQYckIIls,57,67,train
1,abseiling,BZiuKJ0b4sQ,337,347,train
2,abseiling,dQwfb1l-3dM,182,192,train
3,abseiling,WwPL6OqNqB0,133,143,train
4,abseiling,ztuc7tVNUDo,3,13,train


In [ ]:
# Выберем сэмплы, в которых встречается слово "dancing"

df = data[data.label.str.contains('dancing')]

print(data.shape, df.shape)

(536699, 5) (13404, 5)


In [ ]:
# Посмотрим сколько уникальных классов у нас осталось
df.label.nunique()

15

In [ ]:
# Посмотрим на балансировку классов
fig = px.histogram(df, x="label", nbins=len(df["label"].unique()))
fig = fig.update_xaxes(categoryorder='total descending')
fig.update_traces(marker=dict(color='green'))
fig.update_layout(title_text="Distribution of Labels")

fig.show()

In [4]:
# Импорт либ
import os
import random
import time
import warnings
import timm
import av

import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.functional as F
import albumentations as A

from box import Box
from tqdm import tqdm
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision.io import read_video
from albumentations.pytorch.transforms import ToTensorV2

warnings.simplefilter("ignore", UserWarning)

# Тренируем на сете предобученную модель

In [6]:
# Посмотрим на обновленную нашу дату
df = pd.read_csv("data.csv")
df.head()

,Unnamed: 0,name_video,label
0,0,videos/video_0000.mp4,belly dancing
1,1,videos/video_0001.mp4,belly dancing
2,2,videos/video_0002.mp4,belly dancing
3,3,videos/video_0003.mp4,belly dancing
4,4,videos/video_0004.mp4,belly dancing


In [7]:
df.iloc[304]

Unnamed: 0                      304
name_video    videos/video_0304.mp4
label             jumpstyle dancing
Name: 304, dtype: object

In [ ]:
# !ls videos/

In [8]:
# Закодируем лейблы в числовые значения
unique_labels = df['label'].unique()
label_dict = {label: index for index, label in enumerate(unique_labels)}
df['target'] = df.label.map(label_dict)

df.head()

,Unnamed: 0,name_video,label,target
0,0,videos/video_0000.mp4,belly dancing,0
1,1,videos/video_0001.mp4,belly dancing,0
2,2,videos/video_0002.mp4,belly dancing,0
3,3,videos/video_0003.mp4,belly dancing,0
4,4,videos/video_0004.mp4,belly dancing,0


In [9]:
# Cоздадим конфиг для предобученной модели
config = Box()

config.num_workers = 1
config.batch_size = 24
config.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config.seed = 1771
config.model_name = 'mc3_18'
config.num_features = df.target.nunique()
config.optimizer_lr = 0.0001
config.epochs = 10
config.test_size = 0.2

In [10]:
# Напишем датаcэт для наших данных

class DanceDanceDataset(Dataset):
    def __init__(self, df, is_train = False):
        self.df = df
        self.video_path = os.path.abspath("../Colab Notebooks/videos")
        self.is_train = is_train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        target = row['target']

        video_path = os.path.join(self.video_path, row['name_video'])

        video, audio, info = read_video(video_path, pts_unit="sec")
        # Беру только 16 кадров.
        if len(video) > 0:
            if len(video) < 48:
                video = video[:16]
            else:
                video = video[:48:3]
            if self.is_train:
                video = video.numpy()
                video = self.aug(video)
                video = torch.Tensor(video)
            resize_transform = transforms.Resize((112, 112))
            video_resized = torch.stack([resize_transform(frame.permute(2, 0, 1)).permute(1, 2, 0) for frame in video])
            video_normalized = video_resized.permute(3, 0, 1, 2)

            # Лениво нормализую
            tensor_3d = video_normalized / 255
        else:
            tensor_3d = torch.empty(3, 16, 112, 112)

        label = torch.tensor(target).long()
        return tensor_3d, label

In [11]:
# Разобьем наши данные на тест и трейн. cоздадим тренеровочный и тестовый датасэт и даталоадэры
train_df, val_df = train_test_split(df,
                                    test_size=config.test_size,
                                    random_state=config.seed,
                                    stratify=df['target']
                                   )
dataset_train = DanceDanceDataset(train_df.reset_index())
dataset_test = DanceDanceDataset(val_df.reset_index())

train_loader = DataLoader(dataset_train,
                          batch_size=config.batch_size,
                          shuffle=True,
                         )
valid_loader = DataLoader(dataset_test,
                          batch_size=config.batch_size,
                         )

In [ ]:
# Загрузка предобученной модели mc3_18
model = models.video.mc3_18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, config.num_features)
model.to(config.device)

Downloading: "https://download.pytorch.org/models/mc3_18-a90a0ba3.pth" to /root/.cache/torch/hub/checkpoints/mc3_18-a90a0ba3.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 35.0MB/s]


VideoResNet(
  (stem): BasicStem(
    (0): Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv3DSimple(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (1):

In [ ]:
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=config.optimizer_lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)

In [ ]:
# Проведем обучение модели. Для корректной работы и для защиты от сбоев будем сохранять модель после каждой эпохи
for epoch_i in range(1, config.epochs + 1):
    start = time.time()

    print(f'---------------------epoch:{epoch_i}/{config.epochs}---------------------')

    # loss
    avg_train_loss = 0
    avg_val_loss = 0
    summa = 0
    ############## Train #############
    model.train()
    train_pbar = tqdm(train_loader, desc="Training")
    for X,y in (train_pbar):
        X_batch = X.to(config.device)
        y_batch = y.to(config.device)

        optimizer.zero_grad()
        res = model.forward(X_batch)

        loss = loss_f(res, y_batch)

        if torch.cuda.is_available():
            train_pbar.set_postfix(gpu_load=f"{torch.cuda.memory_allocated() / 1024 ** 3:.2f}GB",
                                   loss=f"{loss.item():.4f}")
        else:
            train_pbar.set_postfix(loss=f"{loss.item():.4f}")

        loss.backward()
        optimizer.step()
        avg_train_loss += loss * len(y_batch)

        del X, res



    ########## VALIDATION ###############
    model.eval()
    valid_pbar = tqdm(valid_loader, desc="Testing")
    with torch.no_grad():
        for X,y in (valid_pbar):
            X_batch = X.to(config.device)
            y_batch = y.to(config.device)

            res = model.forward(X_batch)

            loss = loss_f(res, y_batch)
            avg_val_loss += loss * len(y_batch)
            valid_pbar.set_postfix(loss=f"{loss.item():.4f}")

            res = res.detach().cpu()
            y_batch = y_batch.cpu()

            preds = torch.max(F.softmax(res, dim=1), dim=1)
            correct= torch.eq(preds[1], y_batch)
            summa += torch.sum(correct).item()

            del X, res


    torch.cuda.empty_cache()

    avg_train_loss = avg_train_loss / len(dataset_train)
    avg_val_loss = avg_val_loss / len(dataset_test)

    acc = summa / len(dataset_test)

    print(f'epoch: {epoch_i}, lr_rate {optimizer.param_groups[0]["lr"]}')

    print("loss_train: %0.4f| loss_valid: %0.4f|" % (avg_train_loss, avg_val_loss))
    print(f"metric {acc:.<5g}")

    elapsed_time = time.time() - start
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = int(elapsed_time % 60)
    print(f"Elapsed time: {hours:02d}:{minutes:02d}:{seconds:02d}")
    scheduler.step()
    torch.save(model, f"model_ep_{epoch_i}.pt")

---------------------epoch:1/10---------------------


Testing: 100%|██████████| 5/5 [02:38<00:00, 31.76s/it, loss=2.3324]


epoch: 1, lr_rate 0.0001
loss_train: 2.5659| loss_valid: 2.3468|
metric 0.233333
Elapsed time: 00:13:18
---------------------epoch:2/10---------------------


Testing: 100%|██████████| 5/5 [01:05<00:00, 13.06s/it, loss=2.2029]


epoch: 2, lr_rate 0.0001
loss_train: 1.5346| loss_valid: 2.1260|
metric 0.308333
Elapsed time: 00:05:30
---------------------epoch:3/10---------------------


...

epoch: 8, lr_rate 6.400000000000001e-05
loss_train: 0.1146| loss_valid: 1.8801|
metric 0.408333
Elapsed time: 00:05:27
---------------------epoch:9/10---------------------


Testing: 100%|██████████| 5/5 [01:05<00:00, 13.08s/it, loss=1.9258]


epoch: 9, lr_rate 6.400000000000001e-05
loss_train: 0.0962| loss_valid: 1.8740|
metric 0.383333
Elapsed time: 00:05:26
---------------------epoch:10/10---------------------


Testing: 100%|██████████| 5/5 [01:05<00:00, 13.05s/it, loss=1.9345]

epoch: 10, lr_rate 5.120000000000001e-05
loss_train: 0.0771| loss_valid: 1.8808|
metric 0.4..
Elapsed time: 00:05:24


# Модель на кадрах

In [5]:
df = pd.read_csv("data.csv")
df.head()

,Unnamed: 0,name_video,label
0,0,videos/video_0000.mp4,belly dancing
1,1,videos/video_0001.mp4,belly dancing
2,2,videos/video_0002.mp4,belly dancing
3,3,videos/video_0003.mp4,belly dancing
4,4,videos/video_0004.mp4,belly dancing


In [6]:
unique_labels = df['label'].unique()
label_dict = {label: index for index, label in enumerate(unique_labels)}
df['target'] = df.label.map(label_dict)

df.head()

,Unnamed: 0,name_video,label,target
0,0,videos/video_0000.mp4,belly dancing,0
1,1,videos/video_0001.mp4,belly dancing,0
2,2,videos/video_0002.mp4,belly dancing,0
3,3,videos/video_0003.mp4,belly dancing,0
4,4,videos/video_0004.mp4,belly dancing,0


In [7]:
# Cоздадим конфиг для обучения модели
config = Box()

config.num_workers = 1
config.batch_size = 24
config.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config.seed = 1771
config.model_name = 'tf_efficientnet_b0_ns'
config.num_features = df.target.nunique()
config.optimizer_lr = 0.001
config.epochs = 10
config.test_size = 0.2

Основная идея задания 2 заключается в обучения модели на отдельных кадрах, нужно случайнм образом выбирать кадр из видео, и обрабатывать его как изображение, пытаясь классифицировать действие, которое на нем происходит. То есть мы будем здесь классифицировать отдельные кадры из видео. Для обучения модели попробуем взять сеть tf_efficientnet_b0_ns - это улучшенная версия сети effnet_b0, обученная на большем кол-ве данных и более мощной версии.

In [8]:
class DanceRndImgSet(Dataset):
    def __init__(self, df, is_train = False):
        self.df = df
        self.video_path = os.path.abspath("../Colab Notebooks/videos")
        self.is_train = is_train
        if is_train:
            self.aug =  A.Compose([
                A.Resize(height=224, width=224, always_apply=True),
                A.Rotate([-30,30], p=1),
                A.CoarseDropout(max_height=int(224 * 0.17), max_width=int(224 * 0.17),
                                 min_holes=4, max_holes=9, p=0.7),
                A.GaussianBlur(blur_limit=(3, 7), p=0.05),
                A.RandomSnow(p=0.05),
                A.RandomRain(p=0.05),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],),
                ToTensorV2(),
            ])
        else:
            self.aug =  A.Compose([
                A.Resize(height=224, width=224, always_apply=True),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],),
                ToTensorV2(),
            ])


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        target = row['target']

        video_path = os.path.join(self.video_path, row['name_video'])

        video, audio, info = read_video(video_path, pts_unit="sec")
        # Беру случайный кадр
        if len(video) > 0:
            total_frames = video.shape[0]
            random_frame_index = torch.randint(0, total_frames, (1,)).item()
            random_frame = video[random_frame_index].numpy()
            frame_with_aug = self.aug(image=random_frame)['image']

        else:
            random_frame = torch.randint(0, 256, (244, 244, 3), dtype=torch.uint8).numpy()
            frame_with_aug = self.aug(image=random_frame)['image']

        if frame_with_aug is not None:
            print(f"Frame shape: {frame_with_aug.shape}, Type: {frame_with_aug.dtype}")

        label = torch.tensor(target).long()
        return frame_with_aug, label

In [9]:
# Разобьем наши данные на тест и трейн. cоздадим тренеровочный и тестовый датасэт и даталоадэры
train_df, val_df = train_test_split(df,
                                    test_size=config.test_size,
                                    random_state=config.seed,
                                    stratify=df['target']
                                   )
dataset_train = DanceRndImgSet(train_df.reset_index(),
                                 is_train=True)
dataset_test = DanceRndImgSet(val_df.reset_index())

train_loader = DataLoader(dataset_train,
                          batch_size=config.batch_size,
                          shuffle=True,
                         )
valid_loader = DataLoader(dataset_test,
                          batch_size=config.batch_size,
                         )

In [10]:
model_name = config.model_name
model = timm.create_model(model_name, pretrained=True)
model.classifier = nn.Sequential(
    nn.Linear(model.classifier.in_features, config.num_classes)
)
model.to(config.device)

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          16, eps=

In [11]:
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=config.optimizer_lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)

In [12]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

for epoch_i in range(1, config.epochs + 1):
    start = time.time()

    print(f'---------------------epoch:{epoch_i}/{config.epochs}---------------------')

    # loss
    avg_train_loss = 0
    avg_val_loss = 0
    summa = 0
    ############## Train #############
    model.train()
    train_pbar = tqdm(train_loader, desc="Training")
    for X,y in (train_pbar):
        X_batch = X.to(config.device)
        y_batch = y.to(config.device)

        optimizer.zero_grad()
        res = model.forward(X_batch)

        loss = loss_f(res, y_batch)

        if torch.cuda.is_available():
            train_pbar.set_postfix(gpu_load=f"{torch.cuda.memory_allocated() / 1024 ** 3:.2f}GB",
                                   loss=f"{loss.item():.4f}")
        else:
            train_pbar.set_postfix(loss=f"{loss.item():.4f}")

        loss.backward()
        optimizer.step()
        avg_train_loss += loss * len(y_batch)

        del X, res



    ########## VALIDATION ###############
    model.eval()
    valid_pbar = tqdm(valid_loader, desc="Testing")
    with torch.no_grad():
        for X,y in (valid_pbar):
            X_batch = X.to(config.device)
            y_batch = y.to(config.device)

            res = model.forward(X_batch)

            loss = loss_f(res, y_batch)
            avg_val_loss += loss * len(y_batch)
            valid_pbar.set_postfix(loss=f"{loss.item():.4f}")

            res = res.detach().cpu()
            y_batch = y_batch.cpu()

            preds = torch.max(F.softmax(res, dim=1), dim=1)
            correct= torch.eq(preds[1], y_batch)
            summa += torch.sum(correct).item()

            del X, res


    torch.cuda.empty_cache()

    avg_train_loss = avg_train_loss / len(dataset_train)
    avg_val_loss = avg_val_loss / len(dataset_test)

    acc = summa / len(dataset_test)

    print(f'epoch: {epoch_i}, lr_rate {optimizer.param_groups[0]["lr"]}')

    print("loss_train: %0.4f| loss_valid: %0.4f|" % (avg_train_loss, avg_val_loss))
    print(f"metric {acc:.<5g}")

    elapsed_time = time.time() - start
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = int(elapsed_time % 60)
    print(f"Elapsed time: {hours:02d}:{minutes:02d}:{seconds:02d}")
    scheduler.step()
    torch.save(model, f"model_ep_{epoch_i}.pt")

---------------------epoch:1/10---------------------


Training:   0%|          | 0/20 [00:00<?, ?it/s]

Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), 

Training:   5%|▌         | 1/20 [00:34<10:53, 34.39s/it, gpu_load=2.03GB, loss=2.7286]

Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), 

...

Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), 

Testing:  80%|████████  | 4/5 [00:51<00:12, 12.96s/it, loss=3.7717]

Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
Frame shape: torch.Size([3, 224, 224]), 

Testing: 100%|██████████| 5/5 [01:04<00:00, 12.83s/it, loss=3.1301]

Frame shape: torch.Size([3, 224, 224]), Type: torch.float32
epoch: 10, lr_rate 0.0005120000000000001
loss_train: 0.7132| loss_valid: 2.9858|
metric 0.25.
Elapsed time: 00:05:22
